# Filmproduktion Szenen Management Schedule 2
## Ziel des Notebooks
Dieses Notebook demonstriert, wie man eine Neo4j-Datenbank verwendet, um Filmproduktionsszenen zu verwalten, inklusive Erstellung von Szenen, Relationen und Gewichtungen basierend auf gemeinsamen Ressourcen. Außerdem wird ein Minimal Spanning Tree (MST) generiert, um die Planung von Dreharbeiten zu optimieren.


### Verbindung zu Neo4j Datenbank aufbauen

In [4]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

## Erstellung von Knoten und Kanten

In [5]:
def add_scene(tx, scene_id, location, actors, equipment):
    # Szene-Knoten mit eingebetteten Listen von Schauspielern und Ausrüstungen erstellen
    create_scene_query = """
    MERGE (s:Scene {id: $scene_id})
    ON CREATE SET s.location = $location, s.actors = $actors, s.equipment = $equipment
    """
    tx.run(create_scene_query, scene_id=scene_id, location=location, actors=actors, equipment=equipment)

def add_scene(tx, scene_id, location, actors, equipment):
    # Szene-Knoten mit eingebetteten Listen von Schauspielern und Ausrüstungen erstellen
    create_scene_query = """
    MERGE (s:Scene {id: $scene_id})
    ON CREATE SET s.location = $location, s.actors = $actors, s.equipment = $equipment
    """
    tx.run(create_scene_query, scene_id=scene_id, location=location, actors=actors, equipment=equipment)

def create_edges(tx):
    # Kanten zwischen Szenen erstellen, die gemeinsame Schauspieler, Ausrüstungen oder Zeitpläne haben
    create_edge_query = """
    MATCH (s1:Scene), (s2:Scene)
    WHERE s1.id < s2.id AND
          (ANY(actor IN s1.actors WHERE actor IN s2.actors) OR
           ANY(equipment IN s1.equipment WHERE equipment IN s2.equipment) )           
    MERGE (s1)-[:RELATED]->(s2)
    """
    tx.run(create_edge_query)



## Gewichte setzen von Kanten
Niedrieges Gewicht für stärkere Verbindung -> höhere Priorität für das nacheinenader drehen.

Gewichtung niedriger wenn:
- Anzahl geinsames Equipment
- Anzahl gemeisame Actor

In [6]:
def set_weights(tx):
    set_weights_query = """
    MATCH (n:Scene)-[r:RELATED]->(m:Scene)
    WITH n, m, r,
         [x IN n.actors WHERE x IN m.actors] AS sharedActors,
         [y IN n.equipment WHERE y IN m.equipment] AS sharedEquipment
    SET r.weight = 10 - (size(sharedActors) + size(sharedEquipment))
    RETURN n, r, m
    """
    tx.run(set_weights_query)

## Graphenprojektion 
Erstellung eines projizierten Graphen 

In [7]:
def create_graph_project(tx):
    create_graph_query = """
    CALL gds.graph.project(
      'schedulePlanGraph',
      'Scene',
      {
        RELATED: {
          properties: 'weight',
          orientation: 'UNDIRECTED'
        }
      }
    )
    """
    tx.run(create_graph_query)

## Ausführung Spanning Tree

In [8]:
def spanning_tree(tx, start_node_id):
    spanning_tree_query = """
    MATCH (n:Scene {id: $start_node_id})
    CALL gds.spanningTree.write('schedulePlanGraph', {
      sourceNode: n,
      relationshipWeightProperty: 'weight',
      writeProperty: 'writeWeight',
      writeRelationshipType: 'MINST'
    })
    YIELD preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    RETURN preProcessingMillis, computeMillis, writeMillis, effectiveNodeCount
    """
    tx.run(spanning_tree_query, start_node_id=start_node_id)

## MST Anzeigen

In [9]:
def display_tree(tx, start_node_id):
    tree_edges = []
    display_tree_query = """
    MATCH path = (n:Scene {id: $start_node_id})-[:MINST*]-()
    WITH relationships(path) AS rels
    UNWIND rels AS rel
    WITH DISTINCT rel AS rel
    RETURN startNode(rel).id AS Source, endNode(rel).id AS Destination, rel.writeWeight AS Weight
    """
    results = tx.run(display_tree_query, start_node_id=start_node_id)
    for record in results:
        tree_edges.append((record['Source'], record['Destination'], record['Weight']))
        print(record)
    return tree_edges

## Funktionen ausführen

In [13]:
import json

def load_json_data(file_path, driver):
    with open(file_path, 'r') as file:
        data = json.load(file)
        scenes = data['scenes']
        scene_number = 1
        with driver.session() as session:
            for scene in scenes:
                session.execute_write(add_scene, f"Scene {scene_number}", scene['location'], scene['actors'], scene['equipment'])
                scene_number += 1
            session.execute_write(create_edges)
            session.execute_write(set_weights)
            session.execute_write(create_graph_project)
            session.execute_write(spanning_tree, 'Scene 2')
            session.execute_write(display_tree, 'Scene 2')

# Pfad zur JSON-Datei
json_file_path = '/data/scenes.json'
load_json_data(json_file_path, driver)

# Ressourcen freigeben
driver.close()

<Record Source='Scene 2' Destination='Scene 3' Weight=8.0>
<Record Source='Scene 3' Destination='Scene 4' Weight=9.0>
<Record Source='Scene 2' Destination='Scene 1' Weight=8.0>
<Record Source='Scene 1' Destination='Scene 5' Weight=9.0>


## Prallel drehbare Szenen identifizieren

In [43]:
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

def fetch_isolated_scenes():
    isolated_scenes = []

    with driver.session() as session:
        # Abfrage zum Finden von Szenen ohne ausgehende RELATED-Beziehungen
        result = session.run("""
            MATCH (s:Scene)
            WHERE NOT (s)-[:RELATED]->(:Scene)
            RETURN s.id AS id
        """)
        
        # Sammeln aller Szenenstandorte, die keine Abhängigkeiten haben
        isolated_scenes = [record['id'] for record in result]

    return isolated_scenes

def plan_scenes():
    isolated = fetch_isolated_scenes()
    print("Isolierte Szenen, die parallel gedreht werden könnten:")
    for scene in isolated:
        print(scene)

plan_scenes()

# Ressourcen freigeben
driver.close()


Isolierte Szenen, die parallel gedreht werden könnten:
Scene 4
Scene 5


## Schedule Plan zusammensetzen

In [74]:


def display_tree(tx, start_node_id):
    tree_edges = []
    display_tree_query = """
    MATCH path = (n:Scene {id: $start_node_id})-[:MINST*]-()
    WITH relationships(path) AS rels
    UNWIND rels AS rel
    WITH DISTINCT rel AS rel
    RETURN startNode(rel).id AS Source, endNode(rel).id AS Destination
    """
    results = tx.run(display_tree_query, start_node_id=start_node_id)
    for record in results:
        tree_edges.append((record['Source'], record['Destination']))
        print(record)
    return tree_edges

def fetch_isolated_scenes():
    isolated_scenes = []
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            WHERE NOT (s)-[:RELATED]->(:Scene)
            RETURN s.id AS id
        """)
        isolated_scenes = [record['id'] for record in result]
    print("Isolierte Szenen:", isolated_scenes)
    return isolated_scenes

def fetch_tree_edges():
    tree_edges = []
    with driver.session() as session:
        tree_edges = session.execute_write(display_tree, 'Scene 2')
    print("Tree edges:", tree_edges)
    return tree_edges

def plan_scenes():
    isolated = fetch_isolated_scenes()
    tree_edges = fetch_tree_edges()

    # Erstellen eines Plans ohne doppelte Destinations
    seen = set()
    plan = []

    for src, dest in tree_edges:
        if src not in seen:
            plan.append(src)
            seen.add(src)
        if dest not in seen:
            plan.append(dest)
            seen.add(dest)

    print("Vorläufiger Plan ohne isolierte Szenen:", plan)

    # Integrieren der isolierten Szenen
    new_plan = []
    inserted_isolated = False

    for scene in plan:
        if scene in isolated and not inserted_isolated:
            new_plan.append(" and ".join(isolated) + " (parallel drehbar)")
            inserted_isolated = True
        elif scene not in isolated:
            new_plan.append(scene)

    print("Endgültiger Plan mit isolierten Szenen:", new_plan)

# Ausführen der Planungsfunktion
plan_scenes()

# Ressourcen freigeben
driver.close()


C:\Users\Noemi\AppData\Local\Temp\ipykernel_27388\3917558573.py:18: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Isolierte Szenen: ['Scene 4', 'Scene 5']
<Record Source='Scene 2' Destination='Scene 3'>
<Record Source='Scene 3' Destination='Scene 4'>
<Record Source='Scene 2' Destination='Scene 1'>
<Record Source='Scene 1' Destination='Scene 5'>
Tree edges: [('Scene 2', 'Scene 3'), ('Scene 3', 'Scene 4'), ('Scene 2', 'Scene 1'), ('Scene 1', 'Scene 5')]
Vorläufiger Plan ohne isolierte Szenen: ['Scene 2', 'Scene 3', 'Scene 4', 'Scene 1', 'Scene 5']
Endgültiger Plan mit isolierten Szenen: ['Scene 2', 'Scene 3', 'Scene 4 and Scene 5 (parallel drehbar)', 'Scene 1']


C:\Users\Noemi\AppData\Local\Temp\ipykernel_27388\3917558573.py:30: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


In [77]:
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

def display_tree(tx, start_node_id):
    tree_edges = []
    display_tree_query = """
    MATCH path = (n:Scene {id: $start_node_id})-[:MINST*]-()
    WITH relationships(path) AS rels
    UNWIND rels AS rel
    WITH DISTINCT rel AS rel
    RETURN startNode(rel).id AS Source, endNode(rel).id AS Destination
    """
    results = tx.run(display_tree_query, start_node_id=start_node_id)
    for record in results:
        tree_edges.append((record['Source'], record['Destination']))
        print(record)
    return tree_edges

def fetch_isolated_scenes():
    isolated_scenes = []
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            WHERE NOT (s)-[:RELATED]->(:Scene)
            RETURN s.id AS id
        """)
        isolated_scenes = [record['id'] for record in result]
    return isolated_scenes

def fetch_tree_edges():
    tree_edges = []
    with driver.session() as session:
        tree_edges = session.execute_write(display_tree, 'Scene 2')
    return tree_edges

def fetch_scene_details():
    scene_details = {}
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            RETURN s.id AS id, s.actors AS actors, s.equipment AS equipment, s.location AS location
        """)
        for record in result:
            scene_details[record['id']] = {
                'actors': record['actors'],
                'equipment': record['equipment'],
                'location': record['location']
            }
    return scene_details

def plan_scenes():
    isolated = fetch_isolated_scenes()
    tree_edges = fetch_tree_edges()
    scene_details = fetch_scene_details()

    # Erstellen eines Plans ohne doppelte Destinations
    seen = set()
    plan = []

    for src, dest in tree_edges:
        if src not in seen:
            plan.append(src)
            seen.add(src)
        if dest not in seen:
            plan.append(dest)
            seen.add(dest)

    # Integrieren der isolierten Szenen
    new_plan = []
    inserted_isolated = False

    for scene in plan:
        if scene in isolated and not inserted_isolated:
            new_plan.append((" and ".join(isolated), "parallel drehbar"))
            inserted_isolated = True
        elif scene not in isolated:
            new_plan.append(scene)

    print("Schedule Plan:")
    for item in new_plan:
        if isinstance(item, tuple):
            scenes = item[0].split(" and ")
            details = [f"{scene} (Actors: {', '.join(scene_details[scene]['actors'])}, Equipment: {', '.join(scene_details[scene]['equipment'])}, Location: {scene_details[scene]['location']})" for scene in scenes]
            print(", ".join(details) + " " + item[1])
        else:
            details = scene_details[item]
            print(f"{item} (Actors: {', '.join(details['actors'])}, Equipment: {', '.join(details['equipment'])}, Location: {details['location']})")

# Ausführen der Planungsfunktion
plan_scenes()

# Ressourcen freigeben
driver.close()


<Record Source='Scene 2' Destination='Scene 3'>
<Record Source='Scene 3' Destination='Scene 4'>
<Record Source='Scene 2' Destination='Scene 1'>
<Record Source='Scene 1' Destination='Scene 5'>
Schedule Plan:
Scene 2 (Actors: Max Müller, Lisa Feld, Equipment: Drone Y, Camera X, Boom Mic B, Location: Uetliberg)
Scene 3 (Actors: Tom Braun, Johanna Schmidt, Lisa Feld, Equipment: Camera Z, Boom Mic B, Location: Bellevue)
Scene 4 (Actors: Nina Kraft, Oscar Wilde, Equipment: Camera W, Gimbal D, Camera Z, Location: Bahnhofstrasse), Scene 5 (Actors: Sara Löwe, Felix Sonne, Equipment: Camera V, Microphone A, Location: ETH Zürich) parallel drehbar
Scene 1 (Actors: Max Müller, Johanna Schmidt, Equipment: Camera X, Microphone A, Location: Zürich Zentrum)


### Schedule Plan basierend auf DFS

In [14]:
from neo4j import GraphDatabase

# Neo4j connection setup
uri = "bolt://localhost:7687"
username = "neo4j"
password = ""
driver = GraphDatabase.driver(uri, auth=(username, password))

def dfs_traversal(tx, start_scene_id):
    dfs_query = """
    MATCH (start:Scene {id: $start_scene_id})
    CALL gds.dfs.stream('schedulePlanGraph', {
        sourceNode: id(start)
    })
    YIELD path
    UNWIND nodes(path) AS node
    RETURN collect(node.id) AS visitOrder
    """
    result = tx.run(dfs_query, start_scene_id=start_scene_id)
    visit_order = []
    for record in result:
        visit_order.extend(record['visitOrder'])
    print("DFS Visit Order:", visit_order)
    return visit_order

def fetch_isolated_scenes():
    isolated_scenes = []
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            WHERE NOT (s)-[:RELATED]->(:Scene)
            RETURN s.id AS id
        """)
        isolated_scenes = [record['id'] for record in result]
    return isolated_scenes

def fetch_scene_details():
    scene_details = {}
    with driver.session() as session:
        result = session.run("""
            MATCH (s:Scene)
            RETURN s.id AS id, s.actors AS actors, s.equipment AS equipment, s.location AS location
        """)
        for record in result:
            scene_details[record['id']] = {
                'actors': record['actors'],
                'equipment': record['equipment'],
                'location': record['location']
            }
    return scene_details

def plan_scenes():
    isolated = fetch_isolated_scenes()
    scene_details = fetch_scene_details()
    
    # DFS Traversal order
    with driver.session() as session:
        dfs_order = session.write_transaction(dfs_traversal, 'Scene 4')

    # Erstellen eines Plans ohne doppelte Destinations
    seen = set()
    plan = []

    for scene in dfs_order:
        if scene not in seen:
            plan.append(scene)
            seen.add(scene)

    # Integrieren der isolierten Szenen
    new_plan = []
    inserted_isolated = False

    for scene in plan:
        if scene in isolated and not inserted_isolated:
            new_plan.append((" and ".join(isolated), "parallel drehbar"))
            inserted_isolated = True
        elif scene not in isolated:
            new_plan.append(scene)

    print("Schedule Plan:")
    for item in new_plan:
        if isinstance(item, tuple):
            scenes = item[0].split(" and ")
            details = [f"{scene} (Actors: {', '.join(scene_details[scene]['actors'])}, Equipment: {', '.join(scene_details[scene]['equipment'])}, Location: {scene_details[scene]['location']})" for scene in scenes]
            print(", ".join(details) + " " + item[1])
        else:
            details = scene_details[item]
            print(f"{item} (Actors: {', '.join(details['actors'])}, Equipment: {', '.join(details['equipment'])}, Location: {details['location']})")

# Ausführen der Planungsfunktion
plan_scenes()

# Ressourcen freigeben
driver.close()


DFS Visit Order: ['Scene 4', 'Scene 3', 'Scene 2', 'Scene 1', 'Scene 5']
Schedule Plan:
Scene 4 (Actors: Nina Kraft, Oscar Wilde, Equipment: Camera W, Gimbal D, Camera Z, Location: Bahnhofstrasse), Scene 5 (Actors: Sara Löwe, Felix Sonne, Equipment: Camera V, Microphone A, Location: ETH Zürich) parallel drehbar
Scene 3 (Actors: Tom Braun, Johanna Schmidt, Lisa Feld, Equipment: Camera Z, Boom Mic B, Location: Bellevue)
Scene 2 (Actors: Max Müller, Lisa Feld, Equipment: Drone Y, Camera X, Boom Mic B, Location: Uetliberg)
Scene 1 (Actors: Max Müller, Johanna Schmidt, Equipment: Camera X, Microphone A, Location: Zürich Zentrum)


C:\Users\Noemi\AppData\Local\Temp\ipykernel_7400\4262777669.py:58: DeprecationWarning: write_transaction has been renamed to execute_write
  dfs_order = session.write_transaction(dfs_traversal, 'Scene 4')
